### CS513 Data Cleansining AirBnB dataset

Step 1: Open Refine Processing: Already Complete
**Step 2: Python Analysis & Cleansing**

Source: https://uofi.app.box.com/s/y7123ku1dpzd4m3126sqvpa2agvv08hv


In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
file = 'AirBNBV2.csv' #file after open-refine step
airbnb_data = pd.read_csv(file, encoding='utf8')

In [4]:
airbnb_data.dtypes

id                                  int64
name                               object
host_id                             int64
host_name 1                        object
host_name 2                        object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [5]:
dropColumns = ['neighbourhood_group']
airbnb_data = airbnb_data.drop([x for x in dropColumns if x in airbnb_data.columns], axis=1)

airbnb_data['host_name 2'] = airbnb_data['host_name 2'].fillna('')


airbnb_data.set_index('id')
airbnb_data.head(5)

,id,name,host_id,host_name 1,host_name 2,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,Hyde Park Walk To Uchicago/ Theological Seminary,2613,Rebecca,,Hyde Park,41.788865,-87.586709,Private room,50,2,137,2018-11-12,2.92,1,163
1,6715,Lincoln Park Oasis Unit 2 Only,15365,Reem,,O'hare,41.929262,-87.660091,Entire home/apt,255,4,93,2018-08-12,0.81,2,303
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,,West Town,41.902895,-87.681822,Entire home/apt,80,2,321,2018-10-29,2.81,1,362
3,9811,Barbara's Hideaway Old Town,33004,At Home Inn,,Lincoln Park,41.917689,-87.637879,Entire home/apt,150,3,33,2018-10-14,0.59,9,350
4,10610,3 Comforts Of Cooperative Living,2140,Lois,Ed,Hyde Park,41.797085,-87.591949,Private room,35,2,31,2018-07-29,0.63,5,179


In [6]:
airbnb_data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,7.594000e+03,7.594000e+03,7594.000000,7594.000000,7594.000000,7594.000000,7594.000000,6698.000000,7594.000000,7594.000000
mean,1.927845e+07,6.662405e+07,41.903008,-87.665745,147.740585,4.355149,35.443771,2.613099,9.295760,169.729128
std,8.045967e+06,6.347290e+07,0.057096,0.039722,258.261766,17.799005,51.791031,2.185901,25.513872,134.173640
min,2.384000e+03,2.140000e+03,41.651558,-87.843687,0.000000,1.000000,0.000000,0.020000,1.000000,0.000000
25%,1.408539e+07,1.260876e+07,41.877135,-87.687371,60.000000,1.000000,4.000000,0.830000,1.000000,49.000000
50%,2.101118e+07,4.494278e+07,41.907135,-87.662312,99.000000,2.000000,16.000000,2.100000,2.000000,151.000000
75%,2.600125e+07,1.081162e+08,41.941543,-87.639197,165.000000,2.000000,45.000000,3.890000,5.000000,327.000000
max,3.004837e+07,2.257737e+08,42.022365,-87.537898,10000.000000,365.000000,525.000000,27.020000,143.000000,365.000000


In [7]:
#Mark where minimum nights are long
airbnb_data["quality_minimum_nights_long"] = False
airbnb_data.loc[airbnb_data["minimum_nights"] >= 300, "quality_minimum_nights_long"] = True

In [8]:
airbnb_data["quality_availability_365"] = False
airbnb_data.loc[airbnb_data["availability_365"] == 0 , "quality_availability_365"] = True

In [9]:
print(airbnb_data.shape)
airbnb_data_subset = airbnb_data[airbnb_data["quality_minimum_nights_long"] == False]
airbnb_data_subset.shape

(7594, 18)


(7584, 18)

In [10]:
def out_iqr(df , column):
    global lower,upper
    q25, q75 = np.quantile(df[column], 0.25), np.quantile(df[column], 0.75)
    # calculate the IQR
    iqr = q75 - q25
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
    # calculate the lower and upper bound value
    lower, upper = q25 - cut_off, q75 + cut_off
    print('The IQR is',iqr)
    print('The lower bound value is', lower)
    print('The upper bound value is', upper)
    # Calculate the number of records below and above lower and above bound value respectively
    df1 = df[df[column] > upper]
    df2 = df[df[column] < lower]
    return print('Total number of outliers are', df1.shape[0]+ df2.shape[0])

In [11]:
out_iqr(airbnb_data,'price')

The IQR is 105.0
The lower bound value is -97.5
The upper bound value is 322.5
Total number of outliers are 492


In [12]:
def out_std(df, column):
    global lower,upper
    # calculate the mean and standard deviation of the data frame
    data_mean, data_std = df[column].mean(), df[column].std()
    # calculate the cutoff value
    cut_off = data_std * 3
    # calculate the lower and upper bound value
    lower, upper = data_mean - cut_off, data_mean + cut_off
    print('The lower bound value is', lower)
    print('The upper bound value is', upper)
    # Calculate the number of records below and above lower and above bound value respectively
    df1 = df[df[column] > upper]
    df2 = df[df[column] < lower]
    return print('Total number of outliers are', df1.shape[0]+ df2.shape[0])

In [13]:
out_std(airbnb_data,'price')
df_outliers = airbnb_data[(airbnb_data['price'] > upper) | (airbnb_data['price'] < lower)]

#Mark where minimum nights are long
airbnb_data["quality_price_outlier"] = False
airbnb_data.loc[airbnb_data["price"] > upper , "quality_price_outlier"] = True
airbnb_data.loc[airbnb_data["price"] < lower , "quality_price_outlier"] = True
airbnb_data.loc[airbnb_data["price"] == 0 , "quality_price_outlier"] = True

df_outliers = airbnb_data[airbnb_data["quality_price_outlier"] == True]
print(df_outliers.shape)
print(airbnb_data.shape)


The lower bound value is -627.0447138034564
The upper bound value is 922.5258831476755
Total number of outliers are 64
(70, 19)
(7594, 19)


In [14]:
airbnb_data
airbnb_data.loc[airbnb_data['name'] == airbnb_data['host_name 1']]

,id,name,host_id,host_name 1,host_name 2,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,quality_minimum_nights_long,quality_availability_365,quality_price_outlier
4333,22289354,Mario,162903486,Mario,,West Town,41.899609,-87.667333,Private room,10,1,0,NaN,NaN,1,0,False,True,False
4391,22528686,22 E. Elm,165418559,22 E. Elm,,Near North Side,41.902867,-87.628250,Entire home/apt,1500,2,7,2018-06-16,0.71,1,147,False,False,True
7227,29401770,Syed,75322076,Syed,,West Ridge,41.994936,-87.700444,Private room,50,2,0,NaN,NaN,1,88,False,False,False


In [15]:
airbnb_data.head(5)

,id,name,host_id,host_name 1,host_name 2,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,quality_minimum_nights_long,quality_availability_365,quality_price_outlier
0,2384,Hyde Park Walk To Uchicago/ Theological Seminary,2613,Rebecca,,Hyde Park,41.788865,-87.586709,Private room,50,2,137,2018-11-12,2.92,1,163,False,False,False
1,6715,Lincoln Park Oasis Unit 2 Only,15365,Reem,,O'hare,41.929262,-87.660091,Entire home/apt,255,4,93,2018-08-12,0.81,2,303,False,False,False
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,,West Town,41.902895,-87.681822,Entire home/apt,80,2,321,2018-10-29,2.81,1,362,False,False,False
3,9811,Barbara's Hideaway Old Town,33004,At Home Inn,,Lincoln Park,41.917689,-87.637879,Entire home/apt,150,3,33,2018-10-14,0.59,9,350,False,False,False
4,10610,3 Comforts Of Cooperative Living,2140,Lois,Ed,Hyde Park,41.797085,-87.591949,Private room,35,2,31,2018-07-29,0.63,5,179,False,False,False


In [16]:
#(Email hidden by Airbnb)
airbnb_data.loc[airbnb_data["host_name 1"] == '(Email hidden by Airbnb)' , "host_name 1"] = 'Unavailable'
#airbnb_data['host_name'].isnull()
airbnb_data.loc[airbnb_data["host_name 1"].isnull(), "host_name 1"] = 'Unavailable'

df_no_name = airbnb_data[airbnb_data["host_name 1"] == 'Unavailable']
df_no_name

,id,name,host_id,host_name 1,host_name 2,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,quality_minimum_nights_long,quality_availability_365,quality_price_outlier
500,4503103,"Clean, Sunny And Amazing Location",20059524,Unavailable,,Armour Square,41.855248,-87.635065,Entire home/apt,110,1,66,2018-10-07,1.53,4,85,False,False,False
501,4503253,Private Bed And Bath Near The Loop And Mccormick,20059524,Unavailable,,Armour Square,41.855353,-87.633123,Private room,53,1,9,2018-10-28,4.29,4,342,False,False,False
502,4503332,Near The Loop And Mccormick Place,20059524,Unavailable,,Armour Square,41.857582,-87.635083,Entire home/apt,125,1,64,2018-10-21,1.52,4,85,False,False,False
503,4503414,Private Bedroom Near The Loop And Mccormick Place,20059524,Unavailable,,Armour Square,41.856434,-87.631344,Private room,33,1,2,2018-10-20,1.46,4,352,False,False,False
1520,12169862,Newly Furnished/ Family Friendly Great Neighbo...,44002119,Unavailable,,Lake View,41.938415,-87.670306,Entire home/apt,235,2,88,2018-11-14,2.86,1,347,False,False,False
3124,19111753,Nice One Bedroom/ Bath In The South Shore Area.,26381069,Unavailable,,South Shore,41.765934,-87.591417,Private room,52,1,11,2018-10-07,0.63,1,365,False,False,False
4916,23919402,Best View Of The Skyline And 10 Minutes To Loop,13324980,Unavailable,,West Town,41.892125,-87.677815,Entire home/apt,250,2,5,2018-06-02,0.72,1,27,False,False,False
5149,24562425,Dorothy's Place Easy Access To Downtown Chicago,185579695,Unavailable,,Austin,41.875610,-87.755100,Entire home/apt,125,1,26,2018-11-04,4.73,1,228,False,False,False


In [17]:
df_nights = airbnb_data[airbnb_data["name"].str.contains('30 Day', case = False)]
df_nights

,id,name,host_id,host_name 1,host_name 2,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,quality_minimum_nights_long,quality_availability_365,quality_price_outlier
60,289689,"30 Day+ Med Student, Interns & Interim Stays.",1376077,Killian,,Edgewater,41.977939,-87.657636,Private room,25,30,56,2018-09-22,0.67,1,216,False,False,False
1294,10280979,The Whimsical Wrigley House Minimum 30 Day Stays,52873868,Joe,,Lake View,41.953394,-87.667415,Entire home/apt,125,365,27,2016-07-05,0.79,1,365,True,False,False


In [18]:
airbnb_data.loc[airbnb_data["name"].str.contains('30 Day', case = False), "minimum_nights"] = 30
df_nights = airbnb_data[airbnb_data["name"].str.contains('30 Day', case = False)]
df_nights

,id,name,host_id,host_name 1,host_name 2,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,quality_minimum_nights_long,quality_availability_365,quality_price_outlier
60,289689,"30 Day+ Med Student, Interns & Interim Stays.",1376077,Killian,,Edgewater,41.977939,-87.657636,Private room,25,30,56,2018-09-22,0.67,1,216,False,False,False
1294,10280979,The Whimsical Wrigley House Minimum 30 Day Stays,52873868,Joe,,Lake View,41.953394,-87.667415,Entire home/apt,125,30,27,2016-07-05,0.79,1,365,True,False,False


In [19]:
def remote_non_utf8(name):
     return re.sub(r'[^\x00-\x7f]',r' ',name)
        
airbnb_data['name'] = airbnb_data['name'].apply(remote_non_utf8)

In [20]:
### Remove special/unwanted characters from text fields

### Define function
def remove_special_characters(df_column,bad_characters_list):
    clean_df_column = df_column
    for bad_char in bad_characters_list:
        clean_df_column = clean_df_column.str.replace(bad_char,' ')
        print("row changes in column " + str(df_column.name) + " after removing character " + str(bad_char) + ": " ,sum(df_column!=clean_df_column))
    clean_df_column = clean_df_column.str.title()
    return clean_df_column

### Run function
bad_chars_lst = ["*","!","?", "(", ")", "-", "_"]
airbnb_data['name'] = remove_special_characters(airbnb_data['name'],bad_chars_lst)
airbnb_data['host_name 1'] = remove_special_characters(airbnb_data['host_name 1'],bad_chars_lst)
airbnb_data['host_name 2'] = remove_special_characters(airbnb_data['host_name 2'],bad_chars_lst)
display(airbnb_data[["name"]].head(20))


row changes in column name after removing character *:  0
row changes in column name after removing character !:  0
row changes in column name after removing character ?:  0
row changes in column name after removing character (:  0
row changes in column name after removing character ):  0
row changes in column name after removing character -:  0
row changes in column name after removing character _:  140
row changes in column host_name 1 after removing character *:  0
row changes in column host_name 1 after removing character !:  1
row changes in column host_name 1 after removing character ?:  25
row changes in column host_name 1 after removing character (:  34
row changes in column host_name 1 after removing character ):  34
row changes in column host_name 1 after removing character -:  40
row changes in column host_name 1 after removing character _:  40
row changes in column host_name 2 after removing character *:  0
row changes in column host_name 2 after removing character !:  0
ro

,name
0,Hyde Park Walk To Uchicago/ Theological Seminary
1,Lincoln Park Oasis Unit 2 Only
2,Tiny Studio Apartment 94 Walk Score
3,Barbara'S Hideaway Old Town
4,3 Comforts Of Cooperative Living
5,The Biddle House #1
6,Chicago Gold Coast 1 Bedroom Condo
7,Lincoln Park Guest House
8,Luxury In Chicago 2Br/ 2Ba / Parking / Bbq
9,Beautifully Furnished 3 Bed/ 1Bath 1


In [21]:
### Collapse consecutive white spaces in the name, host_name, neighbourhood and room_type columns

### Define function
def remove_consecutive_spaces(df_column):
    clean_df_column = df_column.replace('\s\s+', ' ', regex=True)
    print("row changes in column " + str(df_column.name) +": " ,sum(df_column!=clean_df_column))
    return clean_df_column

### Run function
airbnb_data['name'] = remove_consecutive_spaces(airbnb_data['name'])
airbnb_data['host_name 1'] = remove_consecutive_spaces(airbnb_data['host_name 1'])
airbnb_data['host_name 2'] = remove_consecutive_spaces(airbnb_data['host_name 2'])
airbnb_data['neighbourhood'] = remove_consecutive_spaces(airbnb_data['neighbourhood'])
airbnb_data['room_type'] = remove_consecutive_spaces(airbnb_data['room_type'])

row changes in column name:  144
row changes in column host_name 1:  33
row changes in column host_name 2:  1
row changes in column neighbourhood:  0
row changes in column room_type:  0


In [22]:
### Trim all the leading and trailing white spaces in the name, host_name, neighbourhood and room_type columns

### Define function
def trim_head_tail_space(df_column):
    clean_df_column = df_column.str.strip()
    print("row changes in column " + str(df_column.name) +": " ,sum(df_column!=clean_df_column))
    return clean_df_column

### Run function
airbnb_data['name'] = trim_head_tail_space(airbnb_data['name'])
airbnb_data['host_name 1'] = trim_head_tail_space(airbnb_data['host_name 1'])
airbnb_data['host_name 2'] = trim_head_tail_space(airbnb_data['host_name 2'])
airbnb_data['neighbourhood'] = trim_head_tail_space(airbnb_data['neighbourhood'])
airbnb_data['room_type'] = trim_head_tail_space(airbnb_data['room_type'])

row changes in column name:  115
row changes in column host_name 1:  14
row changes in column host_name 2:  0
row changes in column neighbourhood:  0
row changes in column room_type:  0


In [23]:
print(airbnb_data['name'].isna().sum())
airbnb_data['name'] = airbnb_data['name'].fillna('AirBNBListing')
print(airbnb_data['name'].isna().sum())

airbnb_data.loc[airbnb_data["name"] == "", "name"] = 'AirBnBListing'
airbnb_data.loc[airbnb_data["name"] == 'AirBnBListing']

0
0


,id,name,host_id,host_name 1,host_name 2,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,quality_minimum_nights_long,quality_availability_365,quality_price_outlier
2196,15230616,AirBnBListing,96810944,Lin,Peter,Avondale,41.934573,-87.731411,Private room,40,1,12,2018-10-26,0.47,3,268,False,False,False
2203,15257492,AirBnBListing,96810944,Lin,Peter,Avondale,41.935739,-87.730370,Private room,40,1,9,2018-10-11,0.35,3,364,False,False,False
5137,24541340,AirBnBListing,185387095,Heidi,,Armour Square,41.849463,-87.629957,Private room,50,1,19,2018-11-06,2.86,2,353,False,False,False
5182,24631726,AirBnBListing,185387095,Heidi,,Armour Square,41.849957,-87.631954,Private room,70,1,22,2018-11-12,3.42,2,349,False,False,False
6434,27721072,AirBnBListing,209282256,May,,Brighton Park,41.812755,-87.709473,Private room,55,1,2,2018-10-15,1.54,1,159,False,False,False


In [24]:
### Identify rows with too many missing values

### Create missing indicator for features with missing data
for col in airbnb_data.columns:
    missing = airbnb_data[col].isnull()
    num_missing = np.sum(missing)
    if num_missing > 0:  
        airbnb_data['quality_{}_ismissing'.format(col)] = missing

### Based on the indicator, count number of Nan values in each row
ismissing_cols = [col for col in airbnb_data.columns if 'ismissing' in col]
airbnb_data['quality_num_missing'] = airbnb_data[ismissing_cols].sum(axis=1)



In [25]:
#airbnb_data.loc[airbnb_data['name'] == airbnb_data['host_name 1']]
missing_data_df = airbnb_data.loc[airbnb_data['quality_num_missing'] >= 2]
missing_data_df.shape

(895, 22)

In [26]:
missing_data_df

,id,name,host_id,host_name 1,host_name 2,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,quality_minimum_nights_long,quality_availability_365,quality_price_outlier,quality_last_review_ismissing,quality_reviews_per_month_ismissing,quality_num_missing
55,265238,Bohemian Chic In The Heart Of Lincoln Square,1382472,Ani,,North Center,41.960841,-87.684630,Entire home/apt,75,90,0,NaN,NaN,1,211,False,False,False,True,True,2
127,789429,Guest Bedroom With Private Bath,1185573,Graz Anton,,Uptown,41.961926,-87.662377,Private room,99,365,0,NaN,NaN,3,365,True,False,False,True,True,2
128,789670,Spacious Space In Bucktown 10 Min. From The Loop,4161186,Christine,,Logan Square,41.916606,-87.671591,Private room,225,1,0,NaN,NaN,1,365,False,False,False,True,True,2
141,918314,Spacious 4 Bed Wicker Park Condo,3636846,Quinn,,West Town,41.911387,-87.685168,Private room,50,360,0,NaN,NaN,1,362,True,False,False,True,True,2
164,1129741,Large Modern Apartment Near Cta,4778146,Mila,,East Garfield Park,41.882817,-87.720264,Entire home/apt,80,7,0,NaN,NaN,1,0,False,True,False,True,True,2
222,1684266,Room One,3730806,Erick,,Edgewater,41.987856,-87.663905,Private room,99,1,0,NaN,NaN,6,59,False,False,False,True,True,2
258,2117388,Room Two,3730806,Erick,,Edgewater,41.988755,-87.665149,Private room,79,1,0,NaN,NaN,6,64,False,False,False,True,True,2
259,2117401,Room Three,3730806,Erick,,Edgewater,41.988933,-87.664511,Private room,129,1,0,NaN,NaN,6,67,False,False,False,True,True,2
260,2117415,Room Four,3730806,Erick,,Edgewater,41.989272,-87.665785,Private room,129,1,0,NaN,NaN,6,64,False,False,False,True,True,2
261,2117550,Room Five,3730806,Erick,,Edgewater,41.987971,-87.663618,Private room,89,1,0,NaN,NaN,6,68,False,False,False,True,True,2


In [27]:
print(airbnb_data.shape)
df = airbnb_data.loc[(airbnb_data['quality_availability_365'] == False) ]
print(df.shape)
df  = df.loc[df['quality_minimum_nights_long'] == False]

df.shape

(7594, 22)
(6596, 22)


(6586, 22)

In [28]:
airbnb_data = airbnb_data.rename(columns = {'host_name 1':'host_name_primary', 'host_name 2':'host_name_secondary' })

In [29]:
#reviews missing has been determined, so set na values to 0
airbnb_data['reviews_per_month'] = airbnb_data['reviews_per_month'].fillna(value=0)

In [30]:
airbnb_data.to_csv('02_airbnb_step2_complete.csv', index=False)